In [ ]:
!apt update && apt-get install -y libnetcdf-dev libgd-dev
!mkdir -p glmpy
!git clone --depth 1 --filter=blob:none --sparse https://github.com/AquaticEcoDynamics/efi-workshop
!cd efi-workshop && git sparse-checkout set glmpy && cp -r glmpy ../ && cd .. && rm -rf efi-workshop
!curl https://raw.githubusercontent.com/AquaticEcoDynamics/efi-workshop/refs/heads/main/requirements.txt --output requirements.txt
!pip install -r requirements.txt

In [ ]:
import matplotlib.pyplot as plt

from glmpy import plots
from glmpy.example_sims import SparklingSim

In [ ]:
sparkling = SparklingSim()
sparkling.run(time_sim=True, write_log=True)

In [ ]:
nc = plots.NCProfile("sparkling/output/output.nc")
fig, ax = plt.subplots(figsize=(10, 5))
out = nc.plot_var(ax=ax, var="temp")
col_bar = fig.colorbar(out)
col_bar.set_label("Temperature (°C)")